# Generación de Señales Básicas (Estrategia de Trading)

# Obtención de datos externos
'''esto debe ser el inicio de la maquina es la obtencion de los datos

# JALC #

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

%config IPCompleter


# --- CONFIGURACIÓN DE DATOS ---
TICKER = 'NVIDIA'  # ¡Cambia esto por cualquier símbolo de acción que desees!
START_DATE = '2020-01-02'
END_DATE = '2025-10-05'
TEST_SIZE_RATIO = 0.2  # 20% para prueba

# --- 1. DESCARGA Y PREPARACIÓN DE DATOS ---

# Descargar datos históricos de Yahoo Finance
print(f"Descargando datos históricos para {TICKER}...")
df = yf.download(TICKER, start=START_DATE, end=END_DATE)

# Guardar datos en CSV ------------------------------------------------------
TICKER = f'TICKER'[(TICKER, start = START_DATE, end = END_DATE)].csv
df.to_csv(TICKER)
print('datos guardados en:'[TICKER])
# ---------------------------------------------------------------------------

# Ingeniería de Características (Features)
# 1. Indicador de la Media Móvil Simple (SMA)
df['SMA_10'] = df['Close'].rolling(window=10).mean()
# 2. Diferencia entre precios de la jornada
df['Open_Close_Diff'] = df['Close'] - df['Open']
df['High_Low_Diff'] = df['High'] - df['Low']



# Crear la Variable Objetivo (Target) para Clasificación
# Target = 1 si el precio de CIERRE del día siguiente sube, 0 si baja o se mantiene.
# Usamos .shift(-1) para que la predicción del día 't' sea el resultado del día 't+1'.
df['Target'] = np.where(df['Close'].shift(-1) > df['Close'], 1, 0)

# Limpiar valores NaN (causados por SMA y Target.shift)
df = df.dropna()

# Definir las características (X) y la variable objetivo (y)
FEATURES = ['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_10', 'Open_Close_Diff', 'High_Low_Diff']
X = df[FEATURES]
y = df['Target']

# --- 2. ESCALADO Y DIVISIÓN TEMPORAL ---

# Escalar las características (IMPRESCINDIBLE para SVM)+
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, index=X.index, columns=X.columns)

# División temporal (entrenamiento con datos antiguos, prueba con datos recientes)
split_point = int(len(X_scaled) * (1 - TEST_SIZE_RATIO))
X_train = X_scaled[:split_point]
X_test = X_scaled[split_point:]
y_train = y[:split_point]
y_test = y[split_point:]

print(f"Datos de Entrenamiento: {len(X_train)} filas")
print(f"Datos de Prueba: {len(X_test)} filas")

# --- 3. ENTRENAMIENTO DEL MODELO SVM ---

# Crear y entrenar el modelo SVC (Support Vector Classifier)
# El kernel 'rbf' se usa para manejar las complejas relaciones no lineales del mercado.
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
print("\nEntrenando la Máquina SVM (SVC)...")
svm_model.fit(X_train, y_train)

# --- 4. EVALUACIÓN Y SEÑALES DE TRADING ---

# Hacer predicciones sobre el conjunto de prueba
y_pred = svm_model.predict(X_test)

print("\n" + "="*50)
print("             EVALUACIÓN DEL MODELO")
print("="*50)
print(f"Precisión del Modelo (Accuracy): {accuracy_score(y_test, y_pred):.4f}")
print("\nInforme de Clasificación:")
print(classification_report(y_test, y_pred))

# Generación de Señales de Compra/Venta y Timing
df_results = df[split_point:].copy()
df_results['Actual_Target'] = y_test
df_results['Predicted_Target'] = y_pred

# Señal: Si predice 1 (subida), la señal es 'COMPRA' para el día siguiente.
df_results['Signal'] = df_results['Predicted_Target'].apply(lambda x: 'COMPRA' if x == 1 else 'VENTA/HOLD')

# Simulación de Portafolio
# Calculamos el retorno si se sigue la señal
df_results['Next_Day_Close'] = df_results['Close'].shift(-1)
df_results['Daily_Return'] = (df_results['Next_Day_Close'] / df_results['Close']) - 1
df_results['Strategy_Return'] = df_results['Daily_Return'] * df_results['Predicted_Target']

# Retornos Acumulados
df_results['Cumulative_Strategy_Return'] = (1 + df_results['Strategy_Return']).cumprod()

print("\n" + "="*50)
print(f"      SEÑALES DE TRADING Y RENDIMIENTO ({TICKER})")
print("="*50)

# Mostrar las últimas 10 señales de trading
print("Últimas 10 Señales Generadas (Timing: Al Cierre del Día):\n")
print(df_results[['Close', 'Predicted_Target', 'Signal', 'Strategy_Return']].tail(10))

# Resumen de Rendimiento
final_strategy_return = df_results['Cumulative_Strategy_Return'].iloc[-2] # Penúltimo valor (el último debe ser NaN a chaleco)
print(f"\nRetorno Acumulado Total de la Estrategia SVM: {final_strategy_return:.2f}x")
print("\n*La señal 'COMPRA' se genera al cierre del día (timing) y se ejecuta al siguiente día.")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
# --- Paso 1: Cargar y preparar los datos ---
# Supongamos que tienes un archivo CSV con datos de acciones
# Reemplaza 'tus_datos_acciones.csv' con tu ruta de archivo
try:
    df = pd.read_csv('tus_datos_acciones.csv', index_col='Date', parse_dates=True)
except FileNotFoundError:
    print("Error: El archivo 'tus_datos_acciones.csv' no se encontró. Por favor, asegúrate de tener el archivo correcto.")
    # Crea un DataFrame de ejemplo para que el código sea ejecutable, pero necesitas datos reales para un modelo útil
    data = {
        'Open': np.random.rand(100) * 100,
        'High': np.random.rand(100) * 100 + 5,
        'Low': np.random.rand(100) * 100 - 5,
        'Close': np.random.rand(100) * 100,
        'Volume': np.random.randint(10000, 50000, 100)
    }
    dates = pd.date_range(start='2024-01-01', periods=100)
    df = pd.DataFrame(data, index=dates)


# Crear la variable objetivo (Target): 1 si el precio subió al día siguiente, 0 si bajó o se mantuvo
# .shift(-1) desplaza los precios de cierre un día hacia atrás, así comparamos el precio de cierre actual
# con el precio de cierre del día siguiente.
df['Price_Change'] = df['Close'].diff().shift(-1)
df['Target'] = np.where(df['Price_Change'] > 0, 1, 0) # 1: Subió (Buy/Up), 0: Bajó/Mantuvo (Sell/Down)

# Ingeniería de Características (Features Engineering) - Ejemplos sencillos
# Puedes añadir indicadores técnicos más complejos (RSI, MACD, etc.)
df['Open_Close_Diff'] = df['Close'] - df['Open']
df['High_Low_Diff'] = df['High'] - df['Low']
df['SMA_10'] = df['Close'].rolling(window=10).mean() # Media Móvil Simple de 10 días

# Eliminar filas con valores NaN (introducidos por el cálculo de SMA y Target)
df = df.dropna()

# Definir las características (X) y la variable objetivo (y)
features = ['Open', 'High', 'Low', 'Close', 'Volume', 'Open_Close_Diff', 'High_Low_Diff', 'SMA_10']
X = df[features]
y = df['Target']

# Escalar las características (importante para SVM)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, index=X.index, columns=X.columns)


# --- Paso 2: Dividir los datos y entrenar el modelo (Usando validación temporal) ---
# En series de tiempo, es mejor entrenar con datos antiguos y probar con datos más nuevos
split_point = int(len(X_scaled) * 0.8)
X_train = X_scaled[:split_point]
X_test = X_scaled[split_point:]
y_train = y[:split_point]
y_test = y[split_point:]

# Crear y entrenar el modelo SVC
# Utilizamos el kernel 'rbf' (Radial Basis Function) para manejar relaciones no lineales
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm_model.fit(X_train, y_train)


# --- Paso 3: Evaluar el modelo ---
# Hacer predicciones sobre el conjunto de prueba
y_pred = svm_model.predict(X_test)

# Calcular la precisión (accuracy)
accuracy = accuracy_score(y_test, y_pred)
print("--- Resultados de la Máquina SVM ---")
print(f"Precisión del modelo (Accuracy): {accuracy:.4f}")

# Informe de clasificación
print("\nInforme de Clasificación:")
print(classification_report(y_test, y_pred))

# Ampliacion del codigo de arrriba

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# --- (Asume que los Pasos 1 y 2 del código anterior ya se ejecutaron) ---
# X_test, y_test, svm_model, y_pred ya están definidos y listos para usar

# 1. Recuperar las fechas y el precio de cierre real del conjunto de prueba
df_test = df[split_point:].copy()
df_test['Actual_Target'] = y_test
df_test['Predicted_Target'] = y_pred

# 2. Generar las Señales de Compra/Venta y Timing
# La señal de trading es la PREDICCIÓN de la dirección del día siguiente.
# Si el modelo predice 1 (Subida) mañana, la SEÑAL DE COMPRA se genera HOY (en el cierre de X_test).

# Timing: Generamos la señal de Compra/Venta al cierre del día (índice t) para ejecutar al día siguiente (t+1)
df_test['Signal'] = df_test['Predicted_Target'].apply(lambda x: 'COMPRA' if x == 1 else 'VENTA/HOLD')

# Calcular el Retorno de la Estrategia (para un portafolio simulado)
# Usamos el precio de cierre del día siguiente para calcular la ganancia/pérdida
df_test['Next_Day_Close'] = df_test['Close'].shift(-1)
df_test['Daily_Return'] = (df_test['Next_Day_Close'] / df_test['Close']) - 1

# Retorno si se sigue la SEÑAL:
# Si 'Signal' es 'COMPRA' (1), se toma el retorno del día siguiente.
# Si 'Signal' es 'VENTA/HOLD' (0), el retorno es 0 (no se invierte).
df_test['Strategy_Return'] = df_test['Daily_Return'] * df_test['Predicted_Target']

# Calcular los Retornos Acumulados
df_test['Cumulative_Strategy_Return'] = (1 + df_test['Strategy_Return']).cumprod()
df_test['Cumulative_BuyAndHold_Return'] = (1 + df_test['Daily_Return']).cumprod()

print("\n--- Estrategia de Trading (Muestra de Salida) ---")
print(df_test[['Close', 'Actual_Target', 'Predicted_Target', 'Signal', 'Strategy_Return']].head(10))

print("\nRetorno Acumulado de la Estrategia SVM:", df_test['Cumulative_Strategy_Return'].iloc[-1])
print("Retorno Acumulado Buy & Hold:", df_test['Cumulative_BuyAndHold_Return'].iloc[-2]) # El último valor es NaN

In [ ]:
# Añade el indicador de volatilidad del volumen a tus características (Features):
df['Vol_Std'] = df['Volume'].rolling(window=20).std()
#Esta señal te indicaría cuándo la liquidez/volatilidad de la ejecución
#de la acción que monitoreas está fuera de su rango normal.